# Contraste de Hipótesis entre Modelos de Machine Learning en Julia

## Introducción
En este notebook, no pretende ser un manual completo de como realizar un contraste de hipotesis entre diferentes modelos de machine learning. Lo que se puede encontrar es una comparativa de rendimiento de dos modelos. En concreto y para el notebook sea autocontenido se va a emplear una librería bastante conocida para la creación de los modelos como es MLJ y otra que nos dará el apoyo necesario para la ejecución de los contrastes. Tenga en cuenta que lo mismo se puede hacer con Scikit-Learn siempre que se saquen las medidas de rendimiento. 

## Configuración del entorno
Primero de todo vamos a instalar las librerías que nos van a hacer falta


In [ ]:
using Pkg;
Pkg.add(["DataFrames", "CSV", "MLJ", "Random", "HypothesisTests", "Statistics", "DecisionTree", "MLJDecisionTreeInterface", "NearestNeighborModels"])
using DataFrames, CSV, MLJ, Random, HypothesisTests, Statistics


## Cargar y preparar los datos
El siguiente paso es la descarga y definición del problema que se va a emplear, en este caso, se empleará un sencillo problema de clasificación binario.


In [ ]:

url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
data = CSV.File(download(url), header=false) |> DataFrame

# Renombramos las columnas para mayor claridad
rename!(data, [:num_embarazos, :glucosa, :presion_arterial, :pliegue_cutaneo, :insulina, :indice_masa_corporal, :historia_familiar, :edad, :diabetes])
first(data, 5)  # Mostrar las primeras 5 filas del dataset


## División de los datos en entrenamiento y prueba
A continuación se preparan los datos repartiendolos entre entrenamiento y test.


In [ ]:
using Random

Random.seed!(42)
train, test = partition(eachindex(data.diabetes), 0.7, shuffle=true)
X_train = data[train, Not(:diabetes)]
y_train = categorical(data[train, :diabetes])
X_test = data[test, Not(:diabetes)]
y_test = categorical(data[test, :diabetes])

println("El conjunto inicial es ", size(data), " mientras que el conjunto de entrenamiento es ",size(X_train),"->",size(y_train),
    " y el de test es ", size(X_test),"->",size(y_test) 
)


## Entrenamiento de los modelos
En este caso se van a entrenar dos modelos básicos para comparar su rendimiento el KNN y un árbol de decisión.


In [ ]:

DecisionTree = @load DecisionTreeClassifier pkg=DecisionTree
KNN = @load KNNClassifier pkg=NearestNeighborModels

tree_model = DecisionTree(max_depth=5)
knn_model = KNN(K=3)

tree_machine = machine(tree_model, X_train, y_train)
fit!(tree_machine)
knn_machine = machine(knn_model, X_train, y_train)
fit!(knn_machine)

## Evaluación de los modelos
Evaluaremos el rendimiento de ambos modelos en el conjunto de prueba utilizando cross-validation.


In [ ]:
# Cross-validation para Árbol de Decisión
y_pred_tree = predict_mode(tree_machine, X_test)
# Cross-validation para k-NN
y_pred_knn = predict_mode(knn_machine, X_test)


tree_results = y_pred_tree .== y_test
knn_results = y_pred_knn .== y_test

# Medir precisión en el conjunto de prueba
accuracy_tree = mean(tree_results)
accuracy_knn = mean(knn_results)

println("Precisión en el conjunto de prueba del Árbol de Decisión: ", accuracy_tree)
println("Precisión en el conjunto de prueba del k-NN: ", accuracy_knn)

Si bien estos datos están bien, no nos dan una idea de como de mejor es uno de los modelos respecto del otro. Para poder asegurar con unas ciertas garantias tal cosa es necesario hacer un contraste de hipótesis, para lo cual hace falta más de un experimento como en un cross_validation de 10 como en este caso.

In [ ]:
X = data[:, Not(:diabetes)]
y = categorical(data[:, :diabetes])

tree_machine = machine(tree_model, X, y)
knn_machine = machine(tree_model, X, y)

results_tree = evaluate!(tree_machine, resampling=CV(nfolds=5, shuffle=true, rng=42),
          measure= [Accuracy()])

results_knn = evaluate!(knn_machine, resampling=CV(nfolds=5, shuffle=true, rng=42),
          measure= [Accuracy()])

println("Los resultados para cada partición son:
    DT = ", results_tree.per_fold, "
    KNN= ", results_knn. per_fold)

## Contraste de hipótesis

El rendimiento de comparar varios modelos de machine learning utilizando tanto un test t de Student como un test de Mann-Whitney. Usaremos un conjunto de datos de ejemplo y dos modelos de clasificación. El objetivo es determinar si hay una diferencia significativa en el rendimiento de los modelos.

### Test t de Student
Compararemos las medias de las exactitudes de los dos modelos usando un test t.

In [ ]:
using HypothesisTests;
t_test_result = OneSampleTTest(results_tree.per_fold[1], results_knn.per_fold[1])
println("Resultado del test: p-valor = ", pvalue(t_test_result))

En este caso nos diría que ambas medias son iguales. 

### Test de Mann-Whitney
Este test no paramétrico se utiliza para comparar muestras independientes. Compara las medianas y no es paramétrico, es decir no asume la normalidad en las distribuciones. En este caso es aun menos potente en la detecciónd e diferencias pero nos servirá para ver ejemplo


In [ ]:
kruskal_result = ApproximateMannWhitneyUTest(results_tree.per_fold[1], results_knn.per_fold[1])
println("Resultado del test: p-valor = ", pvalue(kruskal_result))